In [24]:
import os 
import struct
import logging
import argparse
import enum

# logging.basicConfig(level=logging.WARNING)  # 正常模式
logging.basicConfig(level=logging.DEBUG)  # 调试模式

In [1]:

PCIE_CONFIG = {
    0:[
        {"frame": 3454, "word":  20, "bit": 26, "data": "1"},
        {"frame": 3454, "word":  20, "bit": 25, "data": "1"},
        {"frame": 3455, "word":  20, "bit": 24, "data": "1"},
        {"frame": 3455, "word":  20, "bit": 21, "data": "1"},
        {"frame": 3475, "word":  21, "bit":  8, "data": "1"},
        {"frame": 3479, "word":  21, "bit":  8, "data": "1"},
        {"frame": 3829, "word":   0, "bit":  2, "data": "1"},
        {"frame": 3829, "word":  22, "bit":  2, "data": "1"},
        {"frame": 3829, "word":  57, "bit":  2, "data": "1"},
        {"frame": 3829, "word":  79, "bit":  2, "data": "1"}
    ],
    1:[
        {"frame": 3454, "word":  20, "bit": 25, "data": "1"},   
        {"frame": 3454, "word":  20, "bit": 22, "data": "1"},   
        {"frame": 3454, "word":  22, "bit": 12, "data": "1"},   
        {"frame": 3462, "word":  22, "bit": 15, "data": "1"},   
        {"frame": 3468, "word":  22, "bit": 15, "data": "1"},   
        {"frame": 3475, "word":  22, "bit":  8, "data": "1"},   
        {"frame": 3479, "word":  22, "bit":  8, "data": "1"},
        {"frame": 3829, "word":   0, "bit":  2, "data": "1"},
        {"frame": 3829, "word":  22, "bit":  2, "data": "1"},
        {"frame": 3829, "word":  57, "bit":  2, "data": "1"},
        {"frame": 3829, "word":  79, "bit":  2, "data": "1"}  
    ],
    2:[
        {"frame": 3454, "word":  20, "bit": 25, "data": "1"},
        {"frame": 3454, "word":  20, "bit": 22, "data": "1"},
        {"frame": 3455, "word":  22, "bit":  7, "data": "1"},
        {"frame": 3462, "word":  22, "bit": 14, "data": "1"},
        {"frame": 3468, "word":  22, "bit": 15, "data": "1"},
        {"frame": 3475, "word":  22, "bit":  0, "data": "1"},
        {"frame": 3479, "word":  22, "bit":  0, "data": "1"}, 
        {"frame": 3829, "word":   0, "bit":  2, "data": "1"},
        {"frame": 3829, "word":  22, "bit":  2, "data": "1"},
        {"frame": 3829, "word":  57, "bit":  2, "data": "1"},
        {"frame": 3829, "word":  79, "bit":  2, "data": "1"}  
    ]
}

In [8]:
PCIE_CHECK = {
    0:[
        {"frame": 3454, "word":  20, "bit": 26, "data": "1"},
        {"frame": 3455, "word":  20, "bit": 24, "data": "1"},
        {"frame": 3455, "word":  20, "bit": 21, "data": "1"},
        {"frame": 3475, "word":  21, "bit":  8, "data": "1"},
        {"frame": 3479, "word":  21, "bit":  8, "data": "1"},
        
    ],
    1:[ 
        {"frame": 3454, "word":  22, "bit": 12, "data": "1"},   
        {"frame": 3462, "word":  22, "bit": 15, "data": "1"},    
        {"frame": 3475, "word":  22, "bit":  8, "data": "1"},   
        {"frame": 3479, "word":  22, "bit":  8, "data": "1"}
    ],
    2:[
        {"frame": 3455, "word":  22, "bit":  7, "data": "1"},
        {"frame": 3462, "word":  22, "bit": 14, "data": "1"},    
        {"frame": 3475, "word":  22, "bit":  0, "data": "1"},
        {"frame": 3479, "word":  22, "bit":  0, "data": "1"}
    ]
}

for index in PCIE_CHECK:
    group = PCIE_CHECK[index]
    for item in group:
        print(item)
    print("==============")

{'frame': 3454, 'word': 20, 'bit': 26, 'data': '1'}
{'frame': 3455, 'word': 20, 'bit': 24, 'data': '1'}
{'frame': 3455, 'word': 20, 'bit': 21, 'data': '1'}
{'frame': 3475, 'word': 21, 'bit': 8, 'data': '1'}
{'frame': 3479, 'word': 21, 'bit': 8, 'data': '1'}
{'frame': 3454, 'word': 22, 'bit': 12, 'data': '1'}
{'frame': 3462, 'word': 22, 'bit': 15, 'data': '1'}
{'frame': 3475, 'word': 22, 'bit': 8, 'data': '1'}
{'frame': 3479, 'word': 22, 'bit': 8, 'data': '1'}
{'frame': 3455, 'word': 22, 'bit': 7, 'data': '1'}
{'frame': 3462, 'word': 22, 'bit': 14, 'data': '1'}
{'frame': 3475, 'word': 22, 'bit': 0, 'data': '1'}
{'frame': 3479, 'word': 22, 'bit': 0, 'data': '1'}


In [22]:
a = [{"a":[1,2,3,4]},
     {"b":123},
     {"c":121233}]
for item in a:
    for key in item:
        print(key)

a
b
c


In [31]:
binary_str = "00000010000000000011111111100101"
address = binary_str[13:18]
print(address)
crc_data_in = address + binary_str
print(crc_data_in)
new_str = "".join([i for i in crc_data_in[::-1]])
print(new_str)

00000
0000000000010000000000011111111100101
1010011111111100000000000100000000000
